In [1]:
import pymysql
import pandas as pd
import numpy as np

In [2]:
conn = None
cur = None

sql = ""

In [3]:
conn = pymysql.connect(host='127.0.0.1', user='root', password='ehgus1319@', db='mydb', charset='utf8' )
cur = conn.cursor()

In [4]:
def search_user_data(conn):
    sql = 'SELECT * FROM user'
    cur.execute(sql)
    results = cur.fetchall()
    df = pd.DataFrame(results, columns=['user_id', 'username', 'password', 'enabled', 'MBTI', 'SEX', 'AGE'])
#     users = df.set_index('id')
    return df;

In [5]:
def search_webtoon_data(conn):
    sql = 'SELECT * FROM webtoon_data'
    cur.execute(sql)
    results = cur.fetchall()
    df = pd.DataFrame(results, columns=['title_id', 'title_name', 'author', 'day', 'genre', 'story', 'link', 'img_src'])
    webtoons = df.set_index('title_id')
    return webtoons;

In [6]:
def search_rating_data(conn):
    sql = 'SELECT * FROM rating'
    cur.execute(sql)
    results = cur.fetchall()
    df = pd.DataFrame(results, columns=['id', 'user_id', 'webtoon_id', 'rating'])
    return df;

In [7]:
search_user_data(conn).head()

,user_id,username,password,enabled,MBTI,SEX,AGE
0,5,1,$2a$10$tw7VkAGfz28t6wJyPdRfgelqFfqYNGLhbKtONIg...,b'\x01',ISTJ,Male,26
1,7,2,$2a$10$BJ.BaHjto.UEUNG7BcC3juHm/shVIMuxQCuJ8D1...,b'\x01',INFP,Male,26
2,9,3,$2a$10$VKuwa1eFx1E48.x2kFWgSue1bNFjPPC9WmGH/T3...,b'\x01',ENFP,Female,23
3,11,4,$2a$10$XTlPnAmR54V.Ji7DjEaj9O4VS9z05XOI1xnkkzz...,b'\x01',ESFJ,Male,22
4,12,5,$2a$10$xkexm4JzxedgEczLIOj6eefytH4Y76ARHz2laXm...,b'\x01',INTJ,Male,23


In [8]:
search_webtoon_data(conn).head()

,title_name,author,day,genre,story,link,img_src
title_id,,,,,,,
21815,히어로메이커,빤쓰,월,"옴니버스, 판타지",왕은 영웅이 되고 싶어하는 공주의 소원을 들어주기로 전격 결심! 공주를 속이고 마치...,https://comic.naver.com/webtoon/list?titleId=2...,https://shared-comic.pstatic.net/thumb/webtoon...
64997,나이트런,김성민,토,"스토리, 판타지",우주력 430년. 성간이동이 가능해져 별과 별을 이동하는 시대\n인간은 괴수와 싸우...,https://comic.naver.com/webtoon/list?titleId=6...,https://shared-comic.pstatic.net/thumb/webtoon...
131385,쿠베라,카레곰,목,"스토리, 판타지",신의 이름을 가진 소녀 쿠베라 리즈. 15세 생일에 외출에서 돌아오다가 초토화되어버...,https://comic.naver.com/webtoon/list?titleId=1...,https://shared-comic.pstatic.net/thumb/webtoon...
318995,갓 오브 하이스쿨,박용제,금,"스토리, 판타지","전국, 전세계 고등학생 중 가장 쎈 녀석을 뽑는 대회가 열린다.\n허구 100% 막...",https://comic.naver.com/webtoon/list?titleId=3...,https://shared-comic.pstatic.net/thumb/webtoon...
400739,에이머,구동인,화,"스토리, 판타지","작은 우연은 역사를 만든다.\n지구를 침공한 이성인 팜킨 일당과, 그들을 막아선 초...",https://comic.naver.com/webtoon/list?titleId=4...,https://shared-comic.pstatic.net/thumb/webtoon...


In [9]:
search_rating_data(conn).head()

,id,user_id,webtoon_id,rating
0,16,5,64997,3
1,17,5,318995,3
2,18,5,570503,5
3,19,5,552960,2
4,20,5,597447,4


In [10]:
user = search_user_data(conn)
webtoons = search_webtoon_data(conn)
ratings = search_rating_data(conn)

#단순 평점 높은순
def recom_webtoon(n_items):
    webtoon_mean = ratings.groupby(['webtoon_id'])['rating'].mean()
    webtoon_sort = webtoon_mean.sort_values(ascending=False)[:n_items]
    recom_webtoons = webtoons.loc[webtoon_sort.index]
    recommendations = recom_webtoons['title_name']
    return recommendations

recom_webtoon(20)

webtoon_id
773524        거래하실래요?
795262           사형소년
717481           일렉시드
724815        아홉수 우리들
784248      내가 키운 S급들
783769     내 남편과 결혼해줘
737628           별이삼샵
739115           앵무살수
780191        선남친 후연애
778963             덴큐
778580           최강부캐
777767     역대급 영지 설계사
776601           광마회귀
746858    이번 생도 잘 부탁해
747271           나노마신
748105           독립일기
751168            상남자
769568       반드시 해피엔딩
765804        A.I. 닥터
790713       대학원 탈출일지
Name: title_name, dtype: object

In [11]:
# users = user.set_index('user_id')
users = user[['user_id', 'MBTI', 'AGE', 'SEX']]
ratings = ratings[['user_id','webtoon_id', 'rating']]

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
x = ratings.copy()
y = ratings['user_id']

user = users.copy()

for i in range(len(user)):
    if 'TJ' in user.loc[i, 'MBTI']:
        user.loc[i, 'MBTI'] = 0.1
    elif 'FJ' in user.loc[i, 'MBTI']:
        user.loc[i, 'MBTI'] = 0.2
    elif 'FP' in user.loc[i, 'MBTI']:
        user.loc[i, 'MBTI'] = 0.3
    elif 'TP' in user.loc[i, 'MBTI']:
        user.loc[i, 'MBTI'] = 0.4    
    
#     if user.loc[i, 'SEX'] == 'Male':
#         user.loc[i, 'SEX'] = 0.5
#     else:
#         user.loc[i, 'SEX'] = 1
        
    if user.loc[i, 'AGE'] / 10 < 2:
        user.loc[i, 'AGE'] = 0.1
    elif user.loc[i, 'AGE'] / 20 < 2:
        user.loc[i, 'AGE'] = 0.2
    elif user.loc[i, 'AGE'] / 30 < 2:
        user.loc[i, 'AGE'] = 0.3
    elif user.loc[i, 'AGE'] / 40 < 2:
        user.loc[i, 'AGE'] = 0.4
    elif user.loc[i, 'AGE'] / 50 < 2:
        user.loc[i, 'AGE'] = 0.5
        
user_matrix = user.pivot(index = 'user_id',
                        columns = 'SEX',
                        values = 'MBTI')


  
x_train, x_test, y_train, y_test = train_test_split(x,y,
                                                    test_size = 0.25)
                                                    
from sklearn.metrics.pairwise import cosine_similarity
rating_matrix = x_train.pivot(index = 'user_id',
                              columns = 'webtoon_id',
                              values = 'rating')

matrix_dummy = rating_matrix.copy().fillna(0) # 원본 손상 방지
user_matrix_dummy = user_matrix.copy().fillna(0)

user_rating_similarity = cosine_similarity(matrix_dummy, matrix_dummy)

user_info_similarity = cosine_similarity(user_matrix_dummy, user_matrix_dummy)

user_rating_similarity = pd.DataFrame(user_rating_similarity,
                               index = rating_matrix.index,
                               columns = rating_matrix.index)
user_info_similarity = pd.DataFrame(user_info_similarity,
                               index = user_matrix.index,
                               columns = user_matrix.index)


user_similarity = user_rating_similarity.add(user_info_similarity)
user_similarity = user_similarity.dropna(how='all')
user_similarity = user_similarity.dropna(how='all', axis=1)
# user_similarity = user_similarity_df.to_numpy()
pd.DataFrame(user_similarity)


user_id,5,7,9,35,36,41,42,43,44,45,...,50,51,52,53,54,55,56,57,58,59
user_id,,,,,,,,,,,,,,,,,,,,,
5,2.000000,1.247466,0.280828,1.306408,1.318959,1.302088,0.353945,1.221531,0.322500,0.030573,...,1.000000,0.039578,1.018013,0.100472,1.000000,0.037840,1.026235,0.180534,1.243204,0.082194
7,1.247466,2.000000,0.000000,1.000000,1.210830,1.000000,0.198915,1.045793,0.099996,0.130344,...,1.000000,0.033747,1.046078,0.000000,1.000000,0.000000,1.149133,0.000000,1.113365,0.048058
9,0.280828,0.000000,2.000000,0.545545,0.220113,0.000000,1.000000,0.000000,1.260998,1.000000,...,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.267860,0.000000,1.167248
35,1.306408,1.000000,0.545545,2.000000,1.365047,1.586846,0.000000,1.000000,0.142386,0.000000,...,1.000000,0.128142,1.000000,0.000000,1.000000,0.000000,1.127412,0.146130,1.037796,0.091241
36,1.318959,1.210830,0.220113,1.365047,2.000000,1.226253,0.141863,1.031570,0.075833,0.104837,...,1.000000,0.074968,1.155305,0.000000,1.159119,0.000000,1.034272,0.058959,1.303726,0.036814
41,1.302088,1.000000,0.000000,1.586846,1.226253,2.000000,0.213980,1.000000,0.155976,0.000000,...,1.000000,0.140372,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.113861,0.000000
42,0.353945,0.198915,1.000000,0.000000,0.141863,0.213980,2.000000,0.000000,1.186904,1.199774,...,0.000000,1.000000,0.080384,1.108887,0.000000,1.000000,0.089199,1.095909,0.177783,1.071861
43,1.221531,1.045793,0.000000,1.000000,1.031570,1.000000,0.000000,2.000000,0.000000,0.000000,...,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.093159,0.000000
44,0.322500,0.099996,1.260998,0.142386,0.075833,0.155976,1.186904,0.000000,2.000000,1.127862,...,0.000000,1.171655,0.000000,1.116722,0.000000,1.000000,0.081275,1.195750,0.102467,1.133137


In [13]:
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

In [14]:
def score(model):
    id_pairs = zip(x_test['user_id'], x_test['webtoon_id']) # 테스트 데이터 간의 user_id와 webtoon_id 페어를 맞춰 튜플화
    y_pred = np.array([model(user,webtoon) for (user,webtoon) in id_pairs]) # 예측값
    print(y_pred)
    y_true = np.array(x_test['rating']) # 테스트데이터의 실제 값
    print(y_true)
    return RMSE(y_true, y_pred)

In [15]:
merged_ratings = pd.merge(x_train, users)
users.set_index('user_id')

MBTI_mean = merged_ratings[['webtoon_id', 'MBTI', 'rating']].groupby(['webtoon_id', 'MBTI'])['rating'].mean()
print(MBTI_mean)

def cf_mbti(user_id, webtoon_id):
    if webtoon_id in rating_matrix.columns:
        MBTI = users.loc[user_id]['MBTI']
        if MBTI in MBTI_mean[webtoon_id].index:
            MBTI_rating = MBTI_mean[webtoon_id][MBTI]
        else:
            MBTI_rating = 3.0
    else:
        MBTI_rating = 3.0
    return MBTI_rating


webtoon_id  MBTI
21815       ESTP    5.0
            INTP    3.0
64997       ESTP    4.0
            INTP    3.0
            ISTJ    3.0
                   ... 
801446      INTP    4.0
801505      ESFP    4.0
            INTP    3.0
801555      INTP    4.0
801998      ISFJ    4.0
Name: rating, Length: 382, dtype: float64


In [16]:
def CF_knn(user_id, webtoon_id, neighbor_size = 0):
    
    if webtoon_id in rating_matrix.columns:
        sim_scores = user_similarity[user_id].copy()
        webtoon_ratings = rating_matrix[webtoon_id].copy()
        none_rating_idx = webtoon_ratings[webtoon_ratings.isnull()].index
        webtoon_ratings = webtoon_ratings.dropna()
        sim_scores = sim_scores.drop(none_rating_idx)

        if neighbor_size == 0:
            mean_rating = np.dot(sim_scores, webtoon_ratings) / sim_scores.sum()

        else:
            if len(sim_scores) > 1:
                neighbor_size = min(neighbor_size, len(sim_scores)) #sim_score보다 neighbor_size가 클 수 있기 때문에 크기를 맞춰줌.
                sim_scores = np.array(sim_scores)
                webtoon_ratings = np.array(webtoon_ratings)
                user_idx = np.argsort(sim_scores) #sim_scores를 오름차순으로 index값을 추출
                sim_scores = sim_scores[user_idx][-neighbor_size:]
                webtoon_ratings = webtoon_ratings[user_idx][-neighbor_size:] #neighbor_size 만큼 추출
                score_sum = sim_scores.sum()
                if score_sum == 0.0:
                    score_sum = 0.1
                mean_rating = np.dot(sim_scores, webtoon_ratings) / sim_scores.sum() # 행렬 곱

            else:
                mean_rating = 3.0
    else:
        mean_rating = 3.0
        
    return mean_rating

In [38]:
def recom_webtoon(user_id, n_items, neighbor_size = 30):
    user_webtoon = rating_matrix.loc[user_id].copy()

    for webtoon in rating_matrix.columns:
        if pd.notnull(user_webtoon.loc[webtoon]):
            user_webtoon.loc[webtoon] = 0 # 해당 웹툰을 이미 평가한 경우는 제외.
    
        else:
            user_webtoon.loc[webtoon] = CF_knn(user_id, webtoon, neighbor_size)

    webtoon_sort = user_webtoon.sort_values(ascending=False)[:n_items]
    recom_webtoon = webtoons.loc[webtoon_sort.index]
    recommendations = recom_webtoon['title_name']
    return recommendations

recom_webtoon(user_id = 5, n_items = 20, neighbor_size = 30)

webtoon_id
717481                일렉시드
746858         이번 생도 잘 부탁해
780191             선남친 후연애
747271                나노마신
769568            반드시 해피엔딩
650305               호랑이형님
730694              초인의 시대
784248           내가 키운 S급들
790713            대학원 탈출일지
739115                앵무살수
776601                광마회귀
725586                  1초
798917    언니, 이번 생엔 내가 왕비야
801035                퍼니게임
801324     민간인 통제구역 - 일급기밀
801555                 언다잉
758150                입학용병
747269           전지적 독자 시점
791258                부캐인생
753478                 데드퀸
Name: title_name, dtype: object

In [27]:
def score(model, neighbor_size = 0):
    id_pairs = zip(x_test['user_id'], x_test['webtoon_id'])
    y_pred = np.array([model(user, webtoon, neighbor_size) for (user, webtoon) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

score(CF_knn, neighbor_size = 30)

1.1316594526142632